In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import yfinance as yf

In [71]:
import statsmodels.api as sm

In [3]:
company_ticker = ["MARUTI.NS", "ULTRACEMCO.NS", "HDFCBANK.NS", "ASIANPAINT.NS", "WHIRLPOOL.NS", "HINDUNILVR.NS", "LT.NS", "NESTLEIND.NS", "TCS.NS", "BEL.NS", "ZEEL.NS", "JSWSTEEL.NS", "RELIANCE.NS", "ADANIENT.NS", "BHARTIARTL.NS", "AXISBANK.NS", "DMART.NS", "WIPRO.NS", "GODREJCP.NS", "TATAMOTORS.NS"]
company_name = ["Maruti Suzuki", "UltraTech Cement", "HDFC Bank", "Asian Paints", "Whirlpool", "Hindustan Unilever", "Larsen & Toubro", "Nestlé", "TCS", "Bharat Electronics", "Zee Entertainment", "JSW Steel Limited", "Reliance", "Adani Enterprises", "Bharti Airtel", "Axis Bank", "Avenue Supermarts", "Wipro", "Godrej", "Tata Motors"]

In [4]:
dat = pd.DataFrame(yf.Ticker(company_ticker[0]).history(period="5y")["Close"])
dat.columns = [company_name[0]]

In [38]:
for i in range(1, len(company_name)):
    dat[company_name[i]] = yf.Ticker(company_ticker[i]).history(period="5y")["Close"]

In [39]:
fourf = pd.read_csv("./Downloads/FourFactors.csv")

In [40]:
fourf['Date'] = pd.to_datetime(fourf['Date'])  
fourf = fourf.set_index(['Date'])

In [41]:
ret = dat.pct_change()[1:]

In [42]:
fourf = fourf.loc[ret.index.intersection(fourf.index)]
ret = ret.loc[ret.index.intersection(fourf.index)]

In [67]:
fourf = fourf.fillna(method='bfill')
ret = ret.fillna(method='bfill')
ret[ret > 1e308] = 0

In [68]:
fourf_train = fourf.loc[ret.index < ret.index[int(len(ret.index)*0.6)]]
fourf_test = fourf.loc[ret.index >= ret.index[int(len(ret.index)*0.6)]]
ret_train = ret.loc[ret.index < ret.index[int(len(ret.index)*0.6)]]
ret_test = ret.loc[ret.index >= ret.index[int(len(ret.index)*0.6)]]

In [79]:
def solve_robust(ret_train, fourf_train, verbose=True):
    p = ret_train.shape[0]
    m = fourf_train.shape[1]
    n = ret_train.shape[1]
    
    from sklearn.linear_model import LinearRegression
    mean0 = []
    V0 = []
    errors = []
    for i in range(ret_train.shape[1]):
        reg = LinearRegression().fit(fourf_train, ret_train.iloc[:,i])
        V0.append(reg.coef_)
        mean0.append(reg.intercept_)
        errors.append(ret_train.iloc[:,i] - reg.predict(fourf_train))
    mean0 = np.array(mean0)
    V0 = np.array(V0).T
    errors = np.array(errors).T
    w = 0.95
    F = np.cov(fourf_train.T)
    B = fourf_train.T.values
    B1 = B @ np.ones(B.shape[1])
    G = B @ B.T - 1/p * np.outer(B1,B1)
    G_root = np.linalg.cholesky(G)
    H = np.linalg.inv(G_root)@ F @ G_root
    A = sm.add_constant(B.T)
    from scipy.stats import f
    s2 = np.sum(np.square(errors),axis=0) / (p - m - 1)
    gam = np.sqrt((m+1) * (np.linalg.inv(A.T @ A))[0][0] * f.ppf(w,m+1,p-m-1) * s2)
    rho = np.sqrt((m+1) * f.ppf(w,m+1,p-m-1) * s2)
    lam, Q = np.linalg.eig(H)
    lam_max = lam.max()
    H_root = np.linalg.cholesky(H)
    wM = Q.T @ H_root @ G_root @ V0
    D_bar = np.diag(s2)
    D_root = np.linalg.cholesky(D_bar)
    
    #define the variables
    import cvxpy as cp
    nu = cp.Variable(1)
    delt = cp.Variable(1)
    phi = cp.Variable(n)
    psi = cp.Variable(n)
    tau = cp.Variable(1)
    t = cp.Variable(m)
    sig = cp.Variable(1)
    #define the constraints
    cons = [
    cp.SOC(1 + delt, cp.hstack([2 * D_root @ phi, 1 - delt])),
    np.ones(n).T @ phi == 1,
    sig >= 0,
    tau >= 0,
    tau + np.ones(m).T @ t <= nu,
    cp.SOC(delt + tau, cp.hstack([2 * rho.T @ psi, sig - tau])),
    sig <= 1 / lam_max]
    cons += [psi[i] >= phi[i] for i in range(n)] + [psi[i] >= -phi[i] for i in range(n)] + [t[i] >= 0 for i in range(m)] + [cp.SOC(1 - sig * lam[i] + t[i], cp.hstack([2 * (wM @ phi)[i], 1 - sig*lam[i] - t[i]])) for i in range(m)]
    #Results
    prob = cp.Problem(cp.Minimize(nu + delt), cons)
    prob.solve(solver='ECOS', abstol=1e-5)
    
    if verbose:
        print("Problem Status: ", prob.status)
        print("Optimal Value: ", prob.value)
        print("Optimal Portfolio: ", phi.value)
  

In [81]:
solve_robust(ret_train,fourf_train)

Problem Status:  optimal
Optimal Value:  0.1437609348578575
Optimal Portfolio:  [2.57892525e-13 2.30119279e-08 2.36365165e-07 7.16920948e-08
 2.76017259e-08 4.84577298e-08 4.26738810e-02 5.66214869e-08
 1.78375076e-08 1.02878218e-08 3.01604272e-09 5.57151860e-09
 3.85252948e-09 2.84879363e-09 6.30327127e-09 3.97165253e-09
 1.53144279e-08 9.57325555e-01 2.64461131e-08 4.43040210e-09]
